CS 28150 Homework 5

Zack Wang

We wish to devise a class for edge-tridiagonal matrices. It is represented by four arrays, easily enough. Multiplication of a ETM A with a vector B is also very easy to compute, as many of the entries in the matrix are zero. We use a looping function that does at most 4 multiplications per row. The test of the multiplication method checks out.

Solving a solution requires a bit more work, but still only four multiplications and four subtractions per row. This allows for the solving to take significantly less time than a brute-force solving, where each cell would have to be checked. The test for solving also matches our expectations, even at larger n (although with worse quality).

In [242]:
import numpy as np
import matplotlib.pyplot as plt
import math
import time

class Edge_Tridiag:
    size = 0
    d = []
    a = []
    b = []
    e = []
    def __init__(self, *args):
        if len(args) == 1:
            self.size = args[0]
            self.a = [1] * (self.size - 1)
            self.d = [2] * self.size
            self.b = [3] * (self.size - 1)
            self.e = [4] * (self.size - 2)
        else : 
            self.a = args[1]
            self.d = args[2]
            self.b = args[3]
            self.e = args[4]
            self.size = args[0]

    def mult(self, v):
        #multiplies the edge tridiagonal with a given vector
        n = self.size
        sol = [0] * n
        #sol[0] = self.d[0] * v[0] + self.a[0] * v[1] + self.e[0] * v[n-1]
        for i in range(n):
            b_term = 0
            a_term = 0
            e_term = 0            
            d_term = self.d[i] * v[i]
            if (i - 1 >= 0):
                b_term = self.b[i-1] * v[i-1]
            if (i < n - 1):
                a_term = self.a[i] * v[i+1]
            if (i < n-2):
                e_term =  self.e[i] * v[n-1]   
            sol[i] = d_term + b_term + a_term + e_term
        return sol

    def solve(self, v):
        n = self.size
        sol = [0] * n
        #forward solving part
        for i in range(n - 2):
            #scale current row
            scale = self.d[i]
            self.d[i] /= scale
            v[i] /= scale
            if (i < n - 1):
                #print("Orig a is", self.a[i])
                self.a[i] /= scale
                #print("New a is", self.a[i])
            if (i < n - 2):
                self.e[i] /= scale

            self.d[i+1] -= (self.a[i] * self.b[i])
            v[i+1] -= (v[i] * self.b[i]) 
            if (i < n - 3): 
                self.e[i+1] -= (self.e[i] * self.b[i])

        #second-to-last loop special
        scale = self.d[n-2]
        self.a[n-2] -= (self.e[n-3] * self.b[n-3])
        self.a[n-2] /= scale
        self.d[n-2] /= scale
        v[n-2] /= scale
        v[n-1] -= (v[n-2] * self.b[n-3])
        #last loop special
        self.d[n-1] -= (self.a[n-2] * self.b[n-2])
        scale = self.d[n-1]
        self.d[n-1] /= scale
        v[n-1] /= scale
        #backward solution
        for j in range(n): #check this
            a_term = 0
            e_term = 0
            if ((n - j) < n):
                a_term = self.a[n - j - 1] * sol[n - j]
            if (n - j) < n - 1:
                e_term = self.e[n - j - 1] * sol[n-1]
            sol[n- j - 1] = v[n- j - 1] - a_term - e_term
        return sol

In [243]:
#Test One: Multiplication
C = Edge_Tridiag(5)
C.d = list(map(lambda x: 4 + .1 * x, range(5)))
C.a = list(map(lambda x: 1 + .01 * (x ** 2) , range(4)))
C.b = list(map(lambda x: 1 - .01 - (x * .03) , range(4)))
C.e = list(map(lambda x: 1 - (x * .05) , range(3)))

In [244]:
#C-Matrix
#4     1     0      0     1

#.99  4.1   1.01    0    .95

#0    .96   4.2   1.04    .9

#0     0    .93    4.3   1.09

#0     0     0      .9    4.4

In [245]:
prod = C.mult([1, 2, 3, 4, 5])
prod

[11.0, 16.97, 23.18, 25.439999999999998, 25.6]

In [246]:
C.solve(prod)

[1.0078005466331463,
 2.005500540531604,
 3.0045473597669186,
 4.008320215794588,
 4.963297272935812]

In [247]:
big = Edge_Tridiag(40000)
big.d = list(map(lambda x: 4 + .1 * (math.sin(x/1000)), range(40000)))
big.a = list(map(lambda x: .2 * (math.sin(x/2000)) , range(39999)))
big.b = list(map(lambda x: .3 * (math.sin(x/3000)) , range(39999)))
big.e = list(map(lambda x: .4 * (math.sin(x/4000)) , range(39998)))
x = list(range(1,40001))
prod2 = big.mult(x)

In [248]:
begin = time.time()
big.solve(prod2)
print(time.time() - begin, 's')

0.06947779655456543 s


In [249]:
big.solve(prod2)

[1.0,
 2.1738990471396487,
 3.34755898222339,
 4.520958127489589,
 5.694096573907037,
 6.866974412473968,
 8.039591734218021,
 9.211948630196126,
 10.384045191494469,
 11.555881509228378,
 12.727457674542281,
 13.898773778609613,
 15.06982991263275,
 16.240626167842926,
 17.411162635500155,
 18.58143940689319,
 19.7514565733394,
 20.92121422618475,
 22.090712456803633,
 23.259951356598954,
 24.428931017001876,
 25.597651529471868,
 26.766112985496598,
 27.934315476591866,
 29.102259094301484,
 30.269943930197297,
 31.437370075878942,
 32.60453762297402,
 33.771446663137795,
 34.93809728805324,
 36.10448958943093,
 37.27062365900898,
 38.43649958855299,
 39.602117469855834,
 40.76747739473788,
 41.93257945504661,
 43.09742374265671,
 44.26201034946992,
 45.426339367415125,
 46.590410888447984,
 47.75422500455118,
 48.91778180773411,
 50.08108139003303,
 51.244123843510636,
 52.406909260256434,
 53.569437732386326,
 54.731709352042635,
 55.893724211394165,
 57.05548240263593,
 58.2169840

Here we see that the solving method, even for a matrix that is 4k x 4k, takes less than 0.1 seconds to evaluate. The solution provided is a bit off, due to the rounding errors of each row, but it maintains a roughly similar shape to the expected solution. 